In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import cv2
import numpy as np
import random


In [3]:
def convolution(image, kernel):
    image = image[0]
    if kernel.ndim ==4:
        kernel = kernel[0,0]
    else:
        kernel = kernel[0]
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    result_height = image_height - kernel_height + 1
    result_width = image_width - kernel_width + 1

    result = np.zeros((result_height, result_width))

    for i in range(result_height):
        for j in range(result_width):
            result[i, j] = np.sum(image[i:i+kernel_height, j:j+kernel_width] * kernel)

    return result

def same_convolution(image, kernel):
    padded_image = np.pad(image, ((0, kernel.shape[0] - 1), (0, kernel.shape[1] - 1)), mode='constant')
    return convolution(padded_image, kernel)

def full_convolution(image, kernel):
    padded_image = np.pad(image, 1, mode='constant')
    padded_image = np.pad(padded_image, 1, mode='constant')
    flipped_kernel = flip_matrix_180(kernel)
    return np.array([convolution(padded_image, flipped_kernel)]) 

def flip_matrix_180(matrix):
    return np.rot90(matrix, 2)

def cross_correlation(matrix1, matrix2):
    return np.sum(np.matmul(matrix1, matrix2))


In [4]:
def mse(predicted, actual):
    return np.mean((predicted - actual)**2)
def mse_prime( predicted,actual):
    return 2*(predicted-actual)/actual.size

In [5]:
def logloss(y_true, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1).mean()

In [6]:
def logloss_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [7]:
class Layer:
    def __init__(self):
        self.inp = None
        self.output = None
    def forward(self,inp):
        pass
    def backward(self,learning_rate,output_grad):
        pass

In [8]:
class Dense_layer(Layer):
    def __init__(self,inp_size,output_size):
        self.weights = np.random.randn(output_size,inp_size)
        self.bias = np.random.randn(output_size,1) 
    def forward(self,inp):
        self.inp_d = inp
        return np.dot(self.weights,self.inp_d) + self.bias
    def backward(self,learning_rate,output_grad):
        weights_grad = np.dot(output_grad,(self.inp_d).T) 
        inp_grad = np.dot((self.weights).T,output_grad)
        self.weights -= learning_rate * weights_grad
        self.bias -= learning_rate * output_grad
        return inp_grad

In [9]:
class activation_layer(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    def forward(self,inp):
        self.inp_a = inp
        return self.activation(self.inp_a)
    def backward(self,learning_rate,output_grad):
        return np.multiply(output_grad,self.activation(self.inp_a))

In [10]:
class ReLU(activation_layer):
    def __init__(self):
        def ReLU(x):
            return x * (x > 0)

        def dReLU(x):
            return 1. * (x > 0)
        
        super().__init__(ReLU, dReLU)

In [11]:
class Sigmoid(activation_layer):
    def __init__(self):
        def sigmoid(x):
            x = np.float128(x) 
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

In [12]:
class Convolutional(Layer):
    def __init__(self, inp_shape, kernel_size, depth):
        inp_depth, inp_height, inp_width = inp_shape
        self.depth = depth
        self.inp_shape = inp_shape
        self.inp_depth = inp_depth
        self.output_shape = (depth, inp_height - kernel_size + 1, inp_width - kernel_size + 1)
        self.kernels_shape = ( depth, inp_depth,kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, inp):
        self.inp_c = inp
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.inp_depth):
                self.output += convolution(self.inp_c, self.kernels)
        return self.output

    def backward(self, learning_rate, output_grad):
        kernels_grad = np.zeros(self.kernels_shape)
        inp_grad = np.zeros(self.inp_shape)

        for i in range(self.depth):
            for j in range(self.inp_depth):
                kernels_grad = convolution(self.inp_c, output_grad)
                inp_grad += full_convolution(output_grad, self.kernels)

        self.kernels -= learning_rate * kernels_grad
        self.biases -= learning_rate * output_grad
        return inp_grad

In [13]:
class Change_shape(Layer):
    def __init__(self, inp_shape, output_shape):
        self.inp_shape = inp_shape
        self.output_shape = output_shape

    def forward(self, inp):
        return np.reshape(inp, self.output_shape)

    def backward(self,learning_rate, output_grad):
        return np.reshape(output_grad, self.inp_shape)

In [14]:
class Max_pool(Layer):
    def __init__(self, kernel_size, stride):
        self.kernel_size = kernel_size
        self.stride = stride
        
    def forward(self, inp):
        
        result_height = inp.shape[1] // self.kernel_size
        result_width = inp.shape[2] // self.kernel_size

        result = np.zeros((result_height, result_width))

        for i in range(result_height):
            for j in range(result_width):
                result[i, j] = np.max(inp[i*self.kernel_size:(i+1)*self.kernel_size, j*self.kernel_size:(j+1)*self.kernel_size])
        print(result)
        return result

    def backward(self,learning_rate, output_grad):
        
        height, width = output_grad.shape
        result_height = height * self.kernel_size
        result_width = width * self.kernel_size

        result = np.zeros((result_height, result_width))

        for i in range(height):
            for j in range(width):
                max_value = output_grad[i, j]
                max_row = i * self.kernel_size
                max_col = j * self.kernel_size
                result[max_row:max_row + self.kernel_size, max_col:max_col + self.kernel_size] = 0
                result[max_row + np.argmax(max_value) // self.kernel_size, max_col + np.argmax(max_value) % self.kernel_size] = max_value

        return result

In [15]:
def load_data(class_0_folder, class_1_folder):
    class_0_images = []
    for filename in os.listdir(class_0_folder):
        if filename.endswith(".jpg") or filename.endswith(".jpg.jpg"):
            image_path = os.path.join(class_0_folder, filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (80, 80))  # Resize the image to a fixed size 
            
            class_0_images.append((np.array([img]), [[0],[1]]))

    class_1_images = []
    for filename in os.listdir(class_1_folder):
        if filename.endswith(".jpg") or filename.endswith(".jpg.jpg"):
            image_path = os.path.join(class_1_folder, filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (80, 80))  # Resize the image to a fixed size 
            
            class_1_images.append((np.array([img]),[[1],[0]]))

    return class_0_images + class_1_images

def preprocess_data(data):
    processed_data = [[image / 255.0, label] for image, label in data]  # Normalize pixel values to the range [0, 1]
 
    return processed_data
    
def load_validation_test_data(test_class_0_folder, test_class_1_folder):
    

    test_data = []
    for class_0_filename in os.listdir(test_class_0_folder):
        if class_0_filename.endswith(".jpg.jpg") or class_0_filename.endswith(".jpg"):
            image_path = os.path.join(test_class_0_folder, class_0_filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (80, 80))  # Resize the image to a fixed size
            
            test_data.append((np.array([img]), [[0],[1]]))  # Label 0 for class 0
    
    for class_1_filename in os.listdir(test_class_1_folder):
        if class_1_filename.endswith(".jpg") or class_1_filename.endswith(".jpg.jpg"):
            image_path = os.path.join(test_class_1_folder, class_1_filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (80, 80))  # Resize the image to a fixed size 
            
            test_data.append((np.array([img]), [[1],[0]]))  # Assign a placeholder label, adjust as needed

    return test_data

def train_cnn(model, loss, loss_prime, xtrain, ytrain, epochs, learning_rate, batch_size):
    """
    Train the CNN model using a simple training loop.
    """
    n_samples = len(xtrain)  # Number of samples in the training set
    for e in range(epochs):
        error = 0
        start = 0
        while start < n_samples:
            # Define end for the current batch
            end = start + batch_size
            x_train = xtrain[start:end]
            y_train = ytrain[start:end]

            # Iterate over each sample in the batch
            for x, y in zip(x_train, y_train):
                # Forward pass
                output = predict(model, x)

                # Calculate loss/error
                error += loss(y, output)

                # Backward pass
                grad = loss_prime(y, output)
                for layer in reversed(model):
                    grad = layer.backward(learning_rate, grad)

            # Move to the next batch
            start += batch_size

        # Compute average error for this epoch
        error /= len(xtrain)
        print(f"Epoch {e + 1}/{epochs}, Loss={error}")

        
            
def predict(network, inp):
    output = inp
    for layer in network:
        output = layer.forward(output)
    return output


def test_cnn(model, test_images, labels):
    """
    Test the CNN model on a set of images.
    """
    predictions = []
    for image in test_images:
        # Forward pass for each test image
        output = model.forward(image)
        predictions.append(output)

    # Compute the mean squared error between predictions and actual labels
    prediction_error = mean_squared_error(predictions, labels)
    
    return prediction_error


if __name__ == "__main__":
    class_0_folder = '/kaggle/input/gender-classification-dataset/Training/female'
    class_1_folder = '/kaggle/input/gender-classification-dataset/Training/male'
    test_class_0_folder = '/kaggle/input/gender-classification-dataset/Validation/female'
    test_class_1_folder = '/kaggle/input/gender-classification-dataset/Validation/male'

    # Load and preprocess training data
    training_data = load_data(class_0_folder, class_1_folder) 
    random.shuffle(training_data)
    
    processed_training_data = preprocess_data(training_data)
    

    # Separate images and labels for training
    train_images, train_labels = zip(*processed_training_data)
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)

    # Create and train the CNN model
    network = [Convolutional((1, 80, 80), 3, 1),Sigmoid(),Change_shape((1, 78, 78), (6084, 1)),Dense_layer(6084, 250),Sigmoid(),Dense_layer(250, 40),Sigmoid(),Dense_layer(40, 2),Sigmoid()]
    train_cnn(network, logloss, logloss_prime, train_images, train_labels, epochs=10, learning_rate=0.00001,batch_size=500)

    # Load test data
    test_data = load_validation_test_data(test_class_0_folder, test_class_1_folder
    )

    random.shuffle(test_data)
    
    processed_test_data = preprocess_data(test_data)
    test_images, test_labels = zip(*processed_test_data)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    predictions = []
    Y = []
    itet = 0
    for x, y in zip(test_images, test_labels):
        output = predict(network, x)
        predictions.append(np.argmax(output))
        Y.append(np.argmax(y))
        itet +=1
        
    def get_accuracy(predictions, Y):
        return np.sum(predictions == Y) / Y.size
    predictions = np.array(predictions)
    Y = np.array(Y)
        
    print("Accuracy: ",get_accuracy(predictions,Y))

Epoch 1/10, Loss=0.371149320840045
Epoch 2/10, Loss=0.3540602113976791
Epoch 3/10, Loss=0.3457493261357331
Epoch 4/10, Loss=0.3609704686108311
Epoch 5/10, Loss=0.3426194442408264
Epoch 6/10, Loss=0.1716366606760628
Epoch 7/10, Loss=0.143090854218041
Epoch 8/10, Loss=0.1798980652981318
Epoch 9/10, Loss=0.0055358963966433
Epoch 10/10, Loss=0.0052096236703542
Accuracy:  0.93


In [16]:
class Logistic_Regression():
    

    def __init__(self, alpha, num_iter):

        self.alpha = alpha
        self.num_iter = num_iter

    def fit(self, X, Y):
        self.m, self.n,self.l = X.shape
        self.w = np.zeros((self.n,2))
        self.b = np.zeros((2,1))
        self.X = X
        self.Y = Y

        for i in range(self.num_iter):
            self.update_weights()

    def update_weights(self):
        for j in range(len(self.X)):
            Y_hat = 1 / (1 + np.exp( - (self.w.T.dot(self.X[j]) + self.b ) ))    
            dw = (1/self.m)*np.dot(self.X[j], (Y_hat - self.Y[j]).T)
            db = (1/self.m)*np.sum(Y_hat - self.Y)


            self.w = self.w - self.alpha * dw
            self.b = self.b - self.alpha * db


    def predict(self, X):
        itet = 0
        predictions =[]
        for i in range(len(X)):
            Y_pred = 1 / (1 + np.exp( - (self.w.T.dot(X[i]) + self.b ) )) 
            Y_pred = np.where( Y_pred > 0.5, 1, 0)
            predictions.append(Y_pred)
            itet +=1
        return np.array(predictions)

In [17]:
lr = Logistic_Regression(0.1,15)
train = []
for i in train_images:
    i = np.reshape(i,(6400,1))                         
    train.append(i)
train = np.array(train) 
lr.fit(train,train_labels)
test = []    
    
random.shuffle(processed_test_data)

test_images, test_labels = zip(*processed_test_data)
test_images = np.array(test_images)
test_labels = np.array(test_labels)    

for i in test_images:
    i = np.reshape(i,(6400,1))                         
    test.append(i)
    
predictions = lr.predict(test)
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size
    
print("Accuracy: ",get_accuracy(predictions,test_labels))

Accuracy:  0.76


In [18]:
#Label encoding
def label_encode(Y):
    y_label = []
    for i in Y:
        if (i == np.array([[0],[1]])).all():
            y_label.append(-1)
        else:
            y_label.append(1)
    return np.array(y_label)

In [19]:
class SVM_classifier():


    def __init__(self, learning_rate, no_of_iterations, lambda_parameter):

        self.learning_rate = learning_rate
        self.no_of_iterations = no_of_iterations
        self.lambda_parameter = lambda_parameter


    def fit(self, X, Y):
        self.m, self.n,self.l  = X.shape
        self.w = np.zeros((self.n,1))
        self.b = 0
        self.X = X
        self.Y = Y

        # implementing Gradient Descent algorithm for Optimization

        for i in range(self.no_of_iterations):
            self.update_weights()


    def update_weights(self):

        y_label = label_encode(self.Y)
        for index, x_i in enumerate(self.X):
            
            condition = y_label[index] * (np.dot(self.w.T,x_i) - self.b) >= 1
            if (condition == True).all():

                dw = 2 * self.lambda_parameter * self.w
                db = 0

            else:

                dw = 2 * self.lambda_parameter * self.w - np.dot(x_i, y_label[index])
                db = y_label[index]


            self.w = self.w - self.learning_rate * dw
            self.b = self.b - self.learning_rate * db



      # predict the label for a given input value
    def predict(self, X):
        itet = 0
        predictions = []
        for i in range(len(X)):
            output = np.dot(self.w.T,X[i]) - self.b 
            predicted_labels = np.sign(output)
            #y_hat = np.where(predicted_labels <= -1, -1, 1)
            if predicted_labels <=-1:
                y_hat = -1
            else:
                y_hat = 1
            predictions.append(y_hat)
            itet +=1

        return np.array(predictions)


In [20]:
svm = SVM_classifier(0.00001,15,1)
train = []
for i in train_images:
    i = np.reshape(i,(6400,1))                         
    train.append(i)
train = np.array(train)
svm.fit(train,train_labels)
test = []
test_data = load_validation_test_data(test_class_0_folder, test_class_1_folder)
random.shuffle(test_data)

processed_test_data = preprocess_data(test_data)
test_images, test_labels = zip(*processed_test_data)
test_images = np.array(test_images)
test_labels = np.array(test_labels)
for i in test_images:
    i = np.reshape(i,(6400,1))                         
    test.append(i)
test = np.array(test)
predictions = svm.predict(test)
test_labels = label_encode(test_labels)
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size
    
print("Accuracy:",get_accuracy(predictions,test_labels))

Accuracy:  0.77
